In [29]:
#import the pandas library
try:
    import pandas as pd
except: 
    !pip install pandas --quiet
    import pandas as pd
#import the sklearn.preprocessing library
try:
    from sklearn.preprocessing import OneHotEncoder
except:
    !pip install scikit-learn --quiet
    from sklearn.preprocessing import OneHotEncoder
#import the pickle library
try:
    import pickle as pkl
except:
    !pip install pickle --quiet
    import pickle as pkl

In [10]:
#load in training data
train_df = pd.read_csv('train.csv')

In [23]:

try:
    from dataprep.eda import create_report
except:
    !pip install dataprep.eda --quiet
    from dataprep.eda import create_report

eda_report = create_report(train_df, title = "Kaggle: Spaceship Titanic EDA report")
pkl.dump(eda_report, 'SpaceshipEDA.pkl')

ERROR: Could not find a version that satisfies the requirement dataprep.eda (from versions: none)
ERROR: No matching distribution found for dataprep.eda


ModuleNotFoundError: No module named 'dataprep'

In [11]:
#intial length of dataset
print(f"intial length of dataset {len(train_df)}",'\n')

#remove columns that are highly unqiue (name and ID)
train_df=train_df.drop(columns=["Name","PassengerId"])

#handle duplicate rows
#check intial number of duplicates
print(f"intial num dups: {train_df.duplicated().sum()}")
#remove duplicates
train_df = train_df.drop_duplicates()
#show the new number of duplicates
print(f"new num dups: {train_df.duplicated().sum()}",'\n')

#handle NaN values
#check intial number of NaN
print(f"intial num NaN:{train_df.isna().sum()}")
#remove NaN values
train_df.dropna(inplace=True)
#show new number of NaN values
print(f"new num NaN:{train_df.isna().sum()}",'\n')

#fix index on dataframe
train_df.reset_index(drop=True)

#new length of dataset
print(f"new length of dataset {len(train_df)}",'\n')

intial length of dataset 8693 

intial num dups: 15
new num dups: 0 

intial num NaN:HomePlanet      201
CryoSleep       217
Cabin           193
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64
new num NaN:HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64 

new length of dataset 6755 



In [12]:
#current dataset (first 5 rows)
train_df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


Transform data

In [15]:
#turns a binary value into a 1 or 0
def to_binary(df,column):
    #set false to be 0 and true to be 1
    mapping = {False: 0, True: 1}
    #change dataframe to reflect the numerical (binary) representation
    df[column] = df[column].map(mapping)
    return df
#binary columns in dataset
binary_columns = [
    "CryoSleep", #turn cryp sleep 1 and 0
    "VIP", #turn vip to 1 and 0
    "Transported" #turn transported into 1 and 0
]
#goes through all binary columns in dataset and changes them to 1 and 0 in are data
for column in binary_columns:
    train_df = to_binary(train_df,column)


In [16]:
#combine money spent across all services into 1 column
#all the luxury services that cost money in the data
luxury_services = [
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck"
]
#added all money spent and create new column "money_spent" 
train_df["money_spent"] = train_df[luxury_services].sum(axis=1)
#remove all of the luxury services columns
train_df = train_df.drop(columns = luxury_services)

In [18]:
#show data after transformation
transformed_df = train_df
transformed_df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,Transported,money_spent
0,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0,0.0
1,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,1,736.0
2,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,0,10383.0
3,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0,5176.0
4,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,1,1091.0


Normalization

In [32]:
#nominal
nominal_columns = [
    "HomePlanet", #onehot encode homeplanet
    "Destination" #onehot encode destination
]
encoder = OneHotEncoder().fit(transformed_df[nominal_columns])
transformed_df.concat(encoder.transform(transformed_df[nominal_columns]))

AttributeError: 'DataFrame' object has no attribute 'concat'

In [ ]:
#tokenize cabin

#numerical
#minmax age
#minmax money_spent